![LOGO](logo.png)
# (eval '(HyTorch Tutorial))
Introduction to PyTorch Meta-Programming Using the Lisp Dialect Hy

Lead Maintainer: [Rafael Zamora-Resendiz](https://github.com/rz4)

**HyTorch** is a Hy (0.16.0) library running Python (3.7) and PyTorch (1.0.1)
for use in rapid low-level development of deep learning (DL) systems as well as
for experiments in DL meta-programming.

##### Table of Contents
1. [Motivation](#s1)
2. [Installation](#s2)
3. [Hy: Lisp Flavored Python](#s3)
4. [Hytorch in Action](#s4)
    1. [Pytorch Models as S-Expressions](#s41)
    2. [S-Expression Threading](#s42)
    3. [Pattern Matching](#s43)
    4. [S-Expression Refactoring](#s44)
    5. [Full Network Training Example](#s45)
5. [Network Analysis and Meta-Analysis Using HyTorch](#s5)
    1. [Fetching Internal Network Components](#s51)
    2. [Probing Networks Using Tests](#s52)
    3. [Loading Foriegn Pytorch Models](#s53)
    4. [Comparing Network Architectures](#s54)
6. [FUTURE:Hyper-Parameter Search Using Genetic Programming]()
    
---

<a name="s1"></a>
## Motivation
The dynamic execution of PyTorch operations allows enough flexibility to change
computational graphs on the fly. This provides an avenue for Hy, a lisp-binding
library for Python, to be used in establishing meta-programming practices in the
field of deep learning.

While the final goal of this project is to build a framework for DL systems to have
access to their own coding, this coding paradigm also shows promise at accelerating the development of new deep learning models while providing significant access to low-torch tensor operations at runtime. A common trend in current DL packages is an abundance of object-oriented abstraction with packages such as Keras. This only reduces transparity to the already black-box nature of NN systems, and makes interpretability and reproducibility of models even more difficult.

In order to better understand NN models and allow for quick iterative design
over novel or esoteric architectures, a deep learning programmer requires access to an
environment that allows low-level definition of tensor graphs and provides methods to quickly access network components for analysis, while still providing a framework to manage large architectures. I believe that the added expressability of Lisp in combination with PyTorch's functional API allows for this type of programming paradigm, and provides DL researchers an extendable framework which cannot be matched by any other abstracted NN packages.

<a name="s2"></a>
## Installation

The current project has been tested using Hy 0.16.0, PyTorch 1.0.1.post2 and
Python 3.7. The following ***Pip*** command can be used to install **HyTorch**:

```
$ pip3 install git+https://github.com/rz4/HyTorch
```
---

<a name="s3"></a>
## Hy: Lisp Flavored Python

"Hy is a dialect of the language Lisp designed to interact with Python by translating expressions into Python's abstract syntax tree (AST). Similar to Clojure's mapping of s-expressions onto the Java virtual machine (JVM), Hy is meant to operate as a transparent Lisp front end to Python. Lisp allows operating on code as data (metaprogramming). Thus, Hy can be used to write domain-specific languages. Hy also allows Python libraries, including the standard library, to be imported and accessed alongside Hy code with a compiling step converting the data structure of both into Python's AST." [Source: Wikipedia]()

I recommend looking over the [Hy documentation](http://docs.hylang.org/en/stable/tutorial.html) as they do a good job showcasing the various features of Hy. In short, Hy provides a Python-friendly Lisp which anyone who knows Python can easy pickup. Plus, you can import any Python code into Hy as well as importing Hy code to Python! Here is just a little tast of how Hy looks like: 

In [1]:
; Lisp-style Comments

;; Define function hello-world
(defn hello-world [name] (print "Hello" name "! It's a great day to be Lisping!"))

;; Evaluate
(hello-world "FooManCHEW")

Hello FooManCHEW ! It's a great day to be Lisping!


[None, None]

In [2]:
; Importing Numpy
(import [numpy :as np])

; Access Attributes Using Dot Notation
(setv x (np.ones '(10 10)))
(print x)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


[None, None, None]

---
<a name="s4"></a>
## HyTorch In Action

<a name="s41"></a>
### Pytorch Models as S-Expressions:

<a name="s42"></a>
### S-Expression Threading:

<a name="s43"></a>
### Pattern Matching:

<a name="s44"></a>
### S-Expression Refactoring:

<a name="s45"></a>
### Full Network Training Example:

---
<a name="s5"></a>
## Network Analysis and Meta-Analysis Using HyTorch

<a name="s51"></a>
### Fetching Internal Network Components:

<a name="s52"></a>
### Probing Networks Using Tests:

<a name="s53"></a>
### Loading Foriegn Pytorch Models:

<a name="s54"></a>
### Comparing Network Architectures: